## 工具调用 (Tool Calling)

在这个notebook中，我们将演示如何让LLM调用外部工具/函数。工具调用是构建AI Agent的核心功能。


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from pydantic import BaseModel, Field
import os
import subprocess
import dotenv

dotenv.load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

os.makedirs("./temp", exist_ok=True)

def write_file(filename: str, content: str) -> str:
    """写入文件到temp目录"""
    try:
        filepath = os.path.join("./temp", filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        return f"文件已成功写入: {filepath}"
    except Exception as e:
        return f"写入文件失败: {str(e)}"

def run_python_file(filename: str) -> str:
    """运行temp目录下的Python文件"""
    try:
        filepath = os.path.join("./temp", filename)
        if not os.path.exists(filepath):
            return f"文件不存在: {filepath}"
        
        result = subprocess.run(
            ["python3", filepath], 
            capture_output=True, 
            text=True,
            cwd="./temp"
        )
        
        output = ""
        if result.stdout:
            output += f"输出:\n{result.stdout}\n"
        if result.stderr:
            output += f"错误:\n{result.stderr}\n"
        if result.returncode != 0:
            output += f"返回码: {result.returncode}\n"
            
        return output if output else "程序运行完成，无输出"
    except Exception as e:
        return f"运行Python文件失败: {str(e)}"

print("工具函数已定义完成")


工具函数已定义完成


### 方法一：使用 Pydantic JSON Schema + 手动解析

第一种方法是定义工具调用的结构化格式，让LLM返回工具调用的JSON，然后手动解析并执行。


In [11]:
from typing import Union, Any, Literal

class WriteFileArgs(BaseModel):
    filename: str = Field(description="文件名")
    content: str = Field(description="文件内容")

# class RunPythonFileArgs(BaseModel):
#     filename: str = Field(description="要运行的Python文件名")

class ToolCall(BaseModel):
    """工具调用"""
    tool_name: Literal["write_file"] = Field(description="要调用的工具名称")
    arguments: WriteFileArgs = Field(description="工具调用的参数")

system_prompt = """
你是一个编程助手。你需要根据用户的任务调用合适的工具。

可用工具：
1. write_file - 写入文件到temp目录
   参数：filename (文件名), content (文件内容)

请严格按照以下格式返回：
- tool_name: "write_file"
- arguments: 相应的参数对象

"""

user_task = "请写一个Python代码来验证一维Normal分布的最大似然估计(MLE)公式，使用模拟数据进行验证"

simple_model = model.with_structured_output(ToolCall)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_task)
]

tool_call = simple_model.invoke(messages)
print(f"工具调用: {tool_call}")

# 手动解析并执行工具
if tool_call.tool_name == "write_file":  # type: ignore
    result = write_file(**tool_call.arguments.model_dump())  # type: ignore
else:
    result = f"未知工具: {tool_call.tool_name}"  # type: ignore

print(f"工具执行结果: {result}")

工具调用: tool_name='write_file' arguments=WriteFileArgs(filename='verify_normal_mle.py', content='#!/usr/bin/env python\n# -*- coding: utf-8 -*-\n\nimport numpy as np\nimport matplotlib.pyplot as plt\n\ndef verify_normal_mle(num_samples=1000, true_mean=5.0, true_std_dev=2.0):\n    """\n    验证一维Normal分布的最大似然估计(MLE)公式。\n\n    参数：\n    num_samples (int): 模拟数据的样本数量。\n    true_mean (float): 真实的正态分布均值。\n    true_std_dev (float): 真实的正态分布标准差。\n    """\n\n    print(f"\\n--- 验证一维Normal分布的MLE公式 ---")\n    print(f"真实参数：均值 = {true_mean:.4f}, 标准差 = {true_std_dev:.4f}")\n\n    # 1. 生成模拟数据\n    np.random.seed(42) # 为了结果的可复现性\n    data = np.random.normal(loc=true_mean, scale=true_std_dev, size=num_samples)\n    print(f"生成了 {num_samples} 个样本数据。")\n\n    # 2. 计算MLE估计值\n    # 均值的MLE估计：样本均值\n    mle_mean = np.mean(data)\n\n    # 方差的MLE估计：(1/N) * sum((xi - mu_mle)^2)\n    # 注意：这里是MLE的方差估计，而不是无偏估计（无偏估计分母是N-1）\n    mle_variance = np.sum((data - mle_mean)**2) / num_samples\n    mle_std_dev = np.sqrt(mle_variance)

In [12]:
tool_call

ToolCall(tool_name='write_file', arguments=WriteFileArgs(filename='verify_normal_mle.py', content='#!/usr/bin/env python\n# -*- coding: utf-8 -*-\n\nimport numpy as np\nimport matplotlib.pyplot as plt\n\ndef verify_normal_mle(num_samples=1000, true_mean=5.0, true_std_dev=2.0):\n    """\n    验证一维Normal分布的最大似然估计(MLE)公式。\n\n    参数：\n    num_samples (int): 模拟数据的样本数量。\n    true_mean (float): 真实的正态分布均值。\n    true_std_dev (float): 真实的正态分布标准差。\n    """\n\n    print(f"\\n--- 验证一维Normal分布的MLE公式 ---")\n    print(f"真实参数：均值 = {true_mean:.4f}, 标准差 = {true_std_dev:.4f}")\n\n    # 1. 生成模拟数据\n    np.random.seed(42) # 为了结果的可复现性\n    data = np.random.normal(loc=true_mean, scale=true_std_dev, size=num_samples)\n    print(f"生成了 {num_samples} 个样本数据。")\n\n    # 2. 计算MLE估计值\n    # 均值的MLE估计：样本均值\n    mle_mean = np.mean(data)\n\n    # 方差的MLE估计：(1/N) * sum((xi - mu_mle)^2)\n    # 注意：这里是MLE的方差估计，而不是无偏估计（无偏估计分母是N-1）\n    mle_variance = np.sum((data - mle_mean)**2) / num_samples\n    mle_std_dev = np.sqrt(mle_varia

### 方法二：使用 Function Calling

第二种方法是使用LangChain的内置 .bind_tools() 功能 (实际上是包装了模型提供商的function calling功能)，这样可以自动处理工具调用的解析和执行。


In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage

# 使用装饰器定义工具，可以提供符合Google格式的docstring
@tool(parse_docstring=True)
def write_file_tool(filename: str, content: str) -> str:
    """写入文件到temp目录，系统自动拼接 "./temp/" 前缀, 例如 filename="test.txt" 则实际写入 "./temp/test.txt"

    Args:
        filename: 文件名
        content: 文件内容

    Returns:
        操作结果
    """
    return write_file(filename, content)

@tool(parse_docstring=True)
def run_python_file_tool(filename: str) -> str:
    """运行temp目录下的Python文件，系统自动拼接 "./temp/" 前缀, 例如 filename="test.py" 则实际运行 "./temp/test.py"

    Args:
        filename: 文件名

    Returns:
        执行结果
    """
    return run_python_file(filename)

tools = [write_file_tool, run_python_file_tool]
llm_with_tools = model.bind_tools(tools)

messages = [
    SystemMessage(content="你是一个编程助手。用户会给你编程任务，你需要使用可用的工具来完成任务。"),
    HumanMessage(content=user_task)
]

response = llm_with_tools.invoke(messages)

# 执行工具调用
tool_calls = getattr(response, 'tool_calls', [])
if tool_calls:
    for tool_call in tool_calls:
        if tool_call['name'] == 'write_file_tool':
            tool_result = write_file_tool.invoke(tool_call['args'])
        elif tool_call['name'] == 'run_python_file_tool':
            tool_result = run_python_file_tool.invoke(tool_call['args'])
        else:
            tool_result = f"未知工具: {tool_call['name']}"
        
        print(f"工具执行结果: {tool_result}")


工具执行结果: 文件已成功写入: ./temp/mle_normal_distribution.py


### 两种方法的比较

#### 方法一：Pydantic JSON Schema + 手动解析
**优点：**
- 完全控制工具调用的流程
- 可以自定义复杂的工具调用逻辑
- 易于调试和理解

**缺点：**
- 需要手动处理工具调用的解析
- 需要自己管理多轮对话状态
- 错误处理复杂
- 代码量较大

#### Function Calling
**优点：**
- 自动处理工具调用的解析和执行
- 内置多轮对话支持
- 代码简洁
- 错误处理更robust
- 与LangChain生态系统无缝集成

**缺点：**
- 对工具调用流程的控制较少
- 依赖框架的实现
- 调试相对困难

### 最佳实践建议

1. 对于结构化输出，当然选择方法一
2. 对于工具调用，优先选择 方法二

### 总结

需要调用工具的时候，
1. 首先无脑选择LangChain的 .bind_tools()
2. 极少数情况，使用各家API的 function calling (tool calling) 功能
3. 几乎不可能出现的情况：自己定义function calling的schema并试图parse
